# UTS Machine Learning Kelompok 4

## Anggota Kelompok :
- Ema Nelvi Saleky ()
- Philemon (232102317)
- Timotius ()

## Deskripsi Project :  

Proyek ini bertujuan untuk mengembangkan dan mengimplementasikan model machine
learning yang dapat memprediksi gangguan tidur berdasarkan faktor kesehatan dan gaya hidup.
Model ini diharapkan mampu memberikan prediksi yang lebih akurat dibandingkan dengan
metode tradisional, serta menghasilkan wawasan baru tentang faktor-faktor utama yang
mempengaruhi gangguan tidur.

# Step 1. Import Dataset & Library Python

## 1. Import Library
Library yang digunakan termasuk :
- Numpy (Pemrosesan data)
- Pandas (Container data)
- Matplotlib & Seaborn (Visualisasi data)
- Sklearn (Algoritma machine learning & metriks penilaian)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

## 2. Import dataset

Data Sleep, Health, and Lifestyle memiliki 400 baris dan 13 kolom yang menjelaskan tentang pola tidur dan kesehatan seseorang. Data ini memiliki fitur-fitur sebagai berikut :  

● Person ID: Nomor identifikasi untuk setiap orang.

● Gender: Jenis kelamin setiap orang.

● Age: Umur setiap orang dalam satuan tahun.

● Occupation: Pekerjaan setiap orang.

● Sleep Duration (hours): Waktu tidur setiap orang dalam satuan jam.

● Quality of Sleep (scale: 1-10): Sebuah rating yang sifatnya subjektif untuk seberapa
nyenyak tidur setiap orang.

● Physical Activity Level (minutes/day): Jumlah waktu berapa lama setiap orang
melakukan aktivitas fisik dalam satuan menit.

● Stress Level (scale: 1-10): Sebuah rating yang sifatnya subjektif untuk seberapa tinggi level stres setiap orang.

● BMI Category: Kategori BMI untuk setiap orang (e.g., Underweight, Normal, Overweight).

● Blood Pressure (systolic/diastolic): Pengukuran tekanan darah untuk setiap orang,
diindikasikan sebagai tekanan sistolik diatas tekanan diastolik.

● Heart Rate (bpm): frekuensi detak jantung saat beristirahat, diukur dalam satuan detak
per menit.

● Daily Steps: Jumlah langkah yang diambil setiap orang per hari.

● Sleep Disorder: Jenis gangguan tidur yang dialami setiap orang (None, Insomnia, Sleep
Apnea).


### Alasan pemilihan data :


In [2]:
dataset = "dataset.csv"

df = pd.read_csv(dataset)
df.head()


,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


# Step 2. Exploratory Data Analytics

# Step 3. Data Preprocessing

# Step 4. Model Training & Tuning

## 1. Train test split

# Step 5. Evaluation Metrix

# Step 6. Result Visualization

# Kesimpulan